Import thư viện và khởi tạo

In [ ]:
# Import thư viện và khởi tạo
import ee
import geemap
import os
import numpy as np
from datetime import datetime

# Khởi tạo Google Earth Engine với project ID
ee.Initialize(project='ee-bonglantrungmuoi')

# Cấu hình thời gian và tham số
start_date = '2022-01-01'
end_date = '2025-01-01'
CLOUD_COVER_THRESHOLD = 20  # Ngưỡng mây tối đa (%)
PROCESSING_LEVEL = 'HARMONIZED'  # Sử dụng Harmonized collection

# Tạo thư mục lưu dữ liệu
output_folder = './sentinel2_data'
os.makedirs(output_folder, exist_ok=True)

print(f"=== CẤU HÌNH TẢI SENTINEL-2 ===")
print(f"- Thời gian: {start_date} đến {end_date}")
print(f"- Ngưỡng mây: {CLOUD_COVER_THRESHOLD}%")
print(f"- Processing level: {PROCESSING_LEVEL}")
print(f"- Thư mục lưu: {output_folder}")
print(f"- Sử dụng: geemap cho file lớn")
print(f"- Mục tiêu: Dữ liệu quang học cho phân tích độ ẩm đất")

In [ ]:
# Đọc shapefile và tạo Sentinel-2 collection
print("=== TẢI SHAPEFILE VÀ TẠO COLLECTION ===")

# Sử dụng shapefile Lâm Đồng từ GEE Asset
lamdong_asset = ee.FeatureCollection('projects/ee-bonglantrungmuoi/assets/lamdong')
lamdong_geometry = lamdong_asset.geometry()

# Hiển thị thông tin về shapefile
print(f"✓ Đã tải shapefile từ GEE Asset")
print(f"  - Số features: {lamdong_asset.size().getInfo()}")

# Tạo Sentinel-2 collection (SR - Surface Reflectance)
if PROCESSING_LEVEL == 'HARMONIZED':
    collection_name = 'COPERNICUS/S2_SR_HARMONIZED'
else:
    collection_name = 'COPERNICUS/S2_SR'

sentinel2_collection = (ee.ImageCollection(collection_name)
                        .filterBounds(lamdong_geometry)
                        .filterDate(start_date, end_date)
                        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_COVER_THRESHOLD))
                        .sort('system:time_start'))

# Thông tin collection
collection_size = sentinel2_collection.size().getInfo()
print(f"✓ Đã tạo Sentinel-2 collection")
print(f"  - Dataset: {collection_name}")
print(f"  - Tổng số ảnh: {collection_size}")

# Thông tin ảnh đầu tiên
if collection_size > 0:
    first_image = ee.Image(sentinel2_collection.first())
    first_date = ee.Date(first_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    first_cloud = first_image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()
    print(f"  - Ảnh đầu tiên: {first_date} (mây: {first_cloud:.1f}%)")
    
    # Thống kê theo năm
    for year in range(2022, 2025):
        year_collection = sentinel2_collection.filter(ee.Filter.calendarRange(year, year, 'year'))
        year_count = year_collection.size().getInfo()
        
        if year_count > 0:
            # Tính mây trung bình
            avg_cloud = year_collection.aggregate_mean('CLOUDY_PIXEL_PERCENTAGE').getInfo()
            print(f"  - Năm {year}: {year_count} ảnh (mây TB: {avg_cloud:.1f}%)")
        else:
            print(f"  - Năm {year}: 0 ảnh")
else:
    print("⚠️ Không tìm thấy ảnh nào!")

# Hiển thị bands có sẵn
if collection_size > 0:
    band_names = first_image.bandNames().getInfo()
    print(f"\n📊 Bands có sẵn ({len(band_names)}):")
    for i, band in enumerate(band_names):
        print(f"  {i+1:2d}. {band}")
    
    # Bands quan trọng cho phân tích độ ẩm đất
    important_bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']  # Blue, Green, Red, NIR, SWIR1, SWIR2
    print(f"\n💡 Bands quan trọng cho độ ẩm đất: {', '.join(important_bands)}")

In [ ]:
# Functions xử lý và tải dữ liệu Sentinel-2
print("=== THIẾT LẬP FUNCTIONS XỬ LÝ VÀ TẢI DỮ LIỆU ===")

def mask_clouds_s2(image):
    """
    Loại bỏ mây và shadow cho Sentinel-2
    """
    qa = image.select('QA60')
    
    # Bits 10 và 11 là cloud và cirrus
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    
    # Mask mây và cirrus
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    
    return image.updateMask(mask)

def add_indices(image):
    """
    Thêm các chỉ số quang phổ cần thiết cho phân tích độ ẩm đất
    """
    # NDVI - Normalized Difference Vegetation Index
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    
    # NDWI - Normalized Difference Water Index
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    
    # LSWI - Land Surface Water Index
    lswi = image.normalizedDifference(['B8', 'B12']).rename('LSWI')
    
    # SAVI - Soil Adjusted Vegetation Index
    L = 0.5  # Soil brightness correction factor
    savi = image.expression(
        '((NIR - RED) / (NIR + RED + L)) * (1 + L)',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'L': L
        }
    ).rename('SAVI')
    
    return image.addBands([ndvi, ndwi, lswi, savi])

def download_monthly_composite(year, month, resolution=10, include_indices=True):
    """
    Tải composite tháng với geemap
    
    Args:
        year: Năm
        month: Tháng  
        resolution: Độ phân giải (m), mặc định 10m
        include_indices: Có tính toán các chỉ số không
    """
    start_month = ee.Date.fromYMD(year, month, 1)
    end_month = start_month.advance(1, 'month')
    
    # Lọc ảnh theo tháng
    monthly_collection = sentinel2_collection.filterDate(start_month, end_month)
    
    # Kiểm tra có dữ liệu không
    count = monthly_collection.size().getInfo()
    if count == 0:
        print(f"❌ Không có dữ liệu cho {year}-{month:02d}")
        return None
    
    # Tính độ mây trung bình
    avg_cloud = monthly_collection.aggregate_mean('CLOUDY_PIXEL_PERCENTAGE').getInfo()
    print(f"📊 Tháng {year}-{month:02d}: {count} ảnh (mây TB: {avg_cloud:.1f}%)")
    
    # Xử lý collection
    processed_collection = monthly_collection.map(mask_clouds_s2)
    
    if include_indices:
        processed_collection = processed_collection.map(add_indices)
    
    # Chọn bands cần thiết
    if include_indices:
        bands = ['NDVI', 'NDWI', 'LSWI', 'SAVI']
    else:
        bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
    
    # Tạo composite median và clip
    composite = (processed_collection
                .select(bands)
                .median()
                .clip(lamdong_geometry))
    
    # Tạo tên file
    suffix = "_indices" if include_indices else "_bands"
    filename = f'S2_Lamdong_{year}_{month:02d}_res{resolution}m{suffix}.tif'
    filepath = os.path.join(output_folder, filename)
    
    try:
        print(f"🔄 Đang tải {filename}...")
        
        # Sử dụng geemap để tải file
        geemap.download_ee_image(
            image=composite,
            filename=filepath,
            scale=resolution,
            region=lamdong_geometry,
            crs='EPSG:4326'
        )
        
        # Kiểm tra file size
        if os.path.exists(filepath):
            file_size = os.path.getsize(filepath) / (1024*1024)  # MB
            print(f"✅ Đã lưu: {filename} ({file_size:.1f} MB)")
            return filepath
        else:
            print(f"❌ Không tạo được file: {filename}")
            return None
        
    except Exception as e:
        print(f"❌ Lỗi tải {year}-{month:02d}: {str(e)}")
        return None

def download_individual_image(index, resolution=10, include_indices=True):
    """
    Tải ảnh riêng lẻ theo index
    
    Args:
        index: Chỉ số ảnh trong collection
        resolution: Độ phân giải (m)
        include_indices: Có tính toán các chỉ số không
    """
    try:
        image = ee.Image(sentinel2_collection.toList(1, index).get(0))
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        cloud_cover = image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()
        
        # Xử lý ảnh
        processed = mask_clouds_s2(image)
        
        if include_indices:
            processed = add_indices(processed)
            bands = ['NDVI', 'NDWI', 'LSWI', 'SAVI']
            suffix = "_indices"
        else:
            bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
            suffix = "_bands"
        
        processed = processed.select(bands).clip(lamdong_geometry)
        
        filename = f'S2_Lamdong_{date}_cloud{cloud_cover:.0f}_res{resolution}m{suffix}.tif'
        filepath = os.path.join(output_folder, filename)
        
        print(f"🔄 Đang tải {filename}...")
        
        geemap.download_ee_image(
            image=processed,
            filename=filepath,
            scale=resolution,
            region=lamdong_geometry,
            crs='EPSG:4326'
        )
        
        if os.path.exists(filepath):
            file_size = os.path.getsize(filepath) / (1024*1024)
            print(f"✅ Đã lưu: {filename} ({file_size:.1f} MB)")
            return filepath
        else:
            print(f"❌ Không tạo được file: {filename}")
            return None
        
    except Exception as e:
        print(f"❌ Lỗi tải ảnh {index}: {str(e)}")
        return None

def estimate_file_size(resolution=10, include_indices=True):
    """Ước tính kích thước file"""
    try:
        sample_image = ee.Image(sentinel2_collection.first())
        pixel_count = (sample_image.select('B2')
                      .reduceRegion(
                          reducer=ee.Reducer.count(),
                          geometry=lamdong_geometry,
                          scale=resolution,
                          maxPixels=1e9
                      ).get('B2').getInfo())
        
        # Ước tính số bands
        num_bands = 4 if include_indices else 6
        
        # Ước tính: bands × 4 bytes/pixel (Float32)
        size_mb = (pixel_count * num_bands * 4) / (1024 * 1024)
        
        print(f"📏 Ước tính kích thước file (độ phân giải {resolution}m):")
        print(f"   - Số pixel: {pixel_count:,}")
        print(f"   - Số bands: {num_bands}")
        if include_indices:
            print(f"   - Bands: NDVI, NDWI, LSWI, SAVI")
        else:
            print(f"   - Bands: B2, B3, B4, B8, B11, B12")
        print(f"   - Kích thước: {size_mb:.1f} MB ({size_mb/1024:.2f} GB)")
        
        return size_mb
        
    except Exception as e:
        print(f"❌ Không thể ước tính: {str(e)}")
        return 0

print("✅ Đã thiết lập functions xử lý và tải dữ liệu Sentinel-2!")
print("💡 Các chỉ số được tính toán (CHỈ 4 CHỈ SỐ CẦN THIẾT):")
print("   - NDVI: Chỉ số thực vật chuẩn hóa")
print("   - NDWI: Chỉ số nước chuẩn hóa")
print("   - LSWI: Chỉ số nước bề mặt đất")
print("   - SAVI: Chỉ số thực vật điều chỉnh đất")

In [ ]:
# TẢI COMPOSITE THÁNG SENTINEL-2 (2022-2024)
print("=== TẢI ẢNH TRUNG BÌNH THÁNG SENTINEL-2 ===")
print("Thời gian: Tháng 1/2022 → Tháng 12/2024")
print("Chỉ số: NDVI, NDWI, LSWI, SAVI")
print("Xử lý: Loại bỏ mây, tính toán chỉ số độ ẩm đất")

# Cấu hình
RESOLUTION = 10  # Độ phân giải 10m
INCLUDE_INDICES = True  # Bao gồm các chỉ số quang phổ
total_months = 3 * 12  # 3 năm × 12 tháng = 36 tháng

# Ước tính kích thước file
print(f"\n🔍 Ước tính kích thước với độ phân giải {RESOLUTION}m:")
estimated_size = estimate_file_size(RESOLUTION, INCLUDE_INDICES)

print(f"📊 Tổng quan:")
print(f"   - Số tháng cần tải: {total_months} tháng")
print(f"   - Kích thước ước tính/file: {estimated_size:.1f} MB")
print(f"   - Tổng dung lượng ước tính: {(estimated_size * total_months):.1f} MB ({(estimated_size * total_months)/1024:.1f} GB)")
print(f"   - Ngưỡng mây: {CLOUD_COVER_THRESHOLD}%")
print(f"   - Chỉ số: NDVI, NDWI, LSWI, SAVI (4 bands)")

# Xác nhận trước khi tải
if estimated_size > 100:  # Nếu file > 100MB
    print(f"\n⚠️  Mỗi file sẽ khá lớn ({estimated_size:.0f} MB)")
    confirm = input("Bạn có muốn tiếp tục? (y/N): ")
    if confirm.lower() != 'y':
        print("❌ Hủy bỏ quá trình tải")
        exit()

print(f"\n🚀 BẮT ĐẦU TẢI {total_months} COMPOSITE THÁNG SENTINEL-2...")

downloaded_files = []
failed_months = []
count = 0

# Tải từng tháng từ 2022-2024
for year in range(2022, 2025):  # 2022, 2023, 2024
    for month in range(1, 13):  # Tháng 1-12
        count += 1
        
        print(f"\n[{count}/{total_months}] Tháng {year}-{month:02d}")
        
        try:
            filepath = download_monthly_composite(year, month, RESOLUTION, INCLUDE_INDICES)
            
            if filepath:
                downloaded_files.append(filepath)
                print(f"✅ Thành công: {os.path.basename(filepath)}")
            else:
                failed_months.append(f"{year}-{month:02d}")
                print(f"❌ Thất bại: {year}-{month:02d}")
                
        except Exception as e:
            failed_months.append(f"{year}-{month:02d}")
            print(f"❌ Lỗi {year}-{month:02d}: {str(e)}")
        
        # Hiển thị tiến trình
        progress = (count / total_months) * 100
        print(f"📈 Tiến trình: {progress:.1f}% ({count}/{total_months})")

# Tổng kết
print(f"\n{'='*50}")
print(f"🎉 HOÀN TẤT QUÁ TRÌNH TẢI SENTINEL-2!")
print(f"{'='*50}")

if downloaded_files:
    total_size = sum(os.path.getsize(f) for f in downloaded_files) / (1024*1024)
    
    print(f"✅ Thành công:")
    print(f"   - Đã tải: {len(downloaded_files)}/{total_months} file")
    print(f"   - Tổng dung lượng: {total_size:.1f} MB ({total_size/1024:.1f} GB)")
    print(f"   - Thư mục: {output_folder}")
    print(f"   - Chỉ số: NDVI, NDWI, LSWI, SAVI")
    
    # Thống kê theo năm
    for year in range(2022, 2025):
        year_files = [f for f in downloaded_files if f"_{year}_" in f]
        if year_files:
            year_size = sum(os.path.getsize(f) for f in year_files) / (1024*1024)
            print(f"   - Năm {year}: {len(year_files)} file ({year_size:.1f} MB)")

if failed_months:
    print(f"\n❌ Thất bại ({len(failed_months)} tháng):")
    for month in failed_months:
        print(f"   - {month}")
    print(f"\n💡 Có thể thử lại các tháng thất bại bằng cách chạy:")
    print(f"   download_monthly_composite(year, month, {RESOLUTION}, {INCLUDE_INDICES})")

print(f"\n📋 Danh sách file đã tải (10 file đầu):")
for i, filepath in enumerate(downloaded_files[:10], 1):
    filename = os.path.basename(filepath)
    size = os.path.getsize(filepath) / (1024*1024)
    print(f"   {i:2d}. {filename} ({size:.1f} MB)")

if len(downloaded_files) > 10:
    print(f"   ... và {len(downloaded_files) - 10} file khác")

print(f"\n🔍 Sử dụng cell tiếp theo để kiểm tra chi tiết và quản lý file!")
print(f"💡 Dữ liệu này phù hợp cho:")
print(f"   - Phân tích thời gian độ ẩm đất")
print(f"   - Theo dõi thay đổi thực vật")
print(f"   - Lập bản đồ độ ẩm đất nông nghiệp")

In [ ]:
# KIỂM TRA VÀ QUẢN LÝ KẾT QUẢ SENTINEL-2
print("=== KIỂM TRA KẾT QUẢ TẢI SENTINEL-2 ===")

def check_downloaded_files():
    """Kiểm tra các file đã tải với thông tin chi tiết"""
    if not os.path.exists(output_folder):
        print("❌ Thư mục dữ liệu chưa tồn tại")
        return []
    
    files = [f for f in os.listdir(output_folder) if f.endswith('.tif')]
    
    if not files:
        print("📂 Thư mục trống - chưa có file nào")
        return []
    
    # Sắp xếp files theo tên
    files.sort()
    
    print(f"📊 Đã tải {len(files)} file Sentinel-2:")
    print(f"📁 Thư mục: {output_folder}")
    
    total_size = 0
    monthly_files = []
    individual_files = []
    
    for i, file in enumerate(files, 1):
        filepath = os.path.join(output_folder, file)
        size = os.path.getsize(filepath) / (1024*1024)  # MB
        total_size += size
        
        # Phân loại file
        if "_indices" in file:
            monthly_files.append((file, size))
        else:
            individual_files.append((file, size))
        
        if i <= 10:  # Hiển thị 10 file đầu
            print(f"   {i:2d}. {file} ({size:.1f} MB)")
    
    if len(files) > 10:
        print(f"   ... và {len(files) - 10} file khác")
    
    print(f"\n📈 Tổng quan:")
    print(f"   - Composite tháng (có chỉ số): {len(monthly_files)} file")
    print(f"   - Ảnh riêng lẻ: {len(individual_files)} file")
    print(f"   - Tổng dung lượng: {total_size:.1f} MB ({total_size/1024:.2f} GB)")
    
    return files

def analyze_temporal_coverage():
    """Phân tích độ phủ thời gian"""
    files = [f for f in os.listdir(output_folder) if f.endswith('.tif') and 'S2_Lamdong' in f]
    
    if not files:
        print("❌ Không có file nào để phân tích")
        return
    
    print(f"📅 Phân tích độ phủ thời gian:")
    
    # Phân tích theo năm và tháng
    coverage = {}
    for file in files:
        if '_indices' in file:
            # Format: S2_Lamdong_2022_01_res10m_indices.tif
            parts = file.split('_')
            if len(parts) >= 4:
                year = int(parts[2])
                month = int(parts[3])
                
                if year not in coverage:
                    coverage[year] = set()
                coverage[year].add(month)
    
    for year in sorted(coverage.keys()):
        months = sorted(coverage[year])
        print(f"   - Năm {year}: {len(months)} tháng ({', '.join(map(str, months))})")
        
        # Tháng bị thiếu
        missing_months = set(range(1, 13)) - coverage[year]
        if missing_months:
            print(f"     Thiếu: {', '.join(map(str, sorted(missing_months)))}")
    
    # Tổng số tháng có dữ liệu
    total_months = sum(len(months) for months in coverage.values())
    print(f"   - Tổng: {total_months}/36 tháng ({total_months/36*100:.1f}%)")

def view_file_info(filename):
    """Xem thông tin chi tiết của file Sentinel-2"""
    filepath = os.path.join(output_folder, filename)
    
    if not os.path.exists(filepath):
        print(f"❌ File không tồn tại: {filename}")
        return
    
    try:
        import rasterio
        
        with rasterio.open(filepath) as src:
            print(f"📄 Thông tin file: {filename}")
            print(f"   - Kích thước: {src.width} x {src.height} pixels")
            print(f"   - Số bands: {src.count}")
            print(f"   - CRS: {src.crs}")
            print(f"   - Bounds: {src.bounds}")
            print(f"   - Data type: {src.dtypes[0]}")
            
            # Hiển thị tên bands
            if src.count == 4:  # Có chỉ số
                band_names = ['NDVI', 'NDWI', 'LSWI', 'SAVI']
                print(f"   - Bands: {', '.join(band_names)}")
            elif src.count == 6:  # Chỉ có bands gốc
                band_names = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
                print(f"   - Bands: {', '.join(band_names)}")
            else:
                band_names = [f'Band_{i}' for i in range(1, src.count + 1)]
                print(f"   - Bands: {', '.join(band_names)}")
            
            # Đọc một phần dữ liệu để thống kê
            sample_size = min(100, src.height, src.width)
            for i, band_name in enumerate(band_names[:src.count], 1):
                sample = src.read(i, window=((0, sample_size), (0, sample_size)))
                valid_data = sample[sample != src.nodata] if src.nodata else sample
                if len(valid_data) > 0:
                    print(f"   - {band_name}: {valid_data.min():.4f} to {valid_data.max():.4f}")
            
    except ImportError:
        print(f"💡 Cài đặt rasterio để xem thông tin chi tiết: pip install rasterio")
        file_size = os.path.getsize(filepath) / (1024*1024)
        print(f"📄 {filename}: {file_size:.1f} MB")
    except Exception as e:
        print(f"❌ Lỗi đọc file: {str(e)}")

def get_cloud_statistics():
    """Thống kê độ mây của dữ liệu đã tải"""
    try:
        # Lấy thống kê độ mây từ collection
        cloud_stats = sentinel2_collection.aggregate_array('CLOUDY_PIXEL_PERCENTAGE').getInfo()
        
        if cloud_stats:
            print(f"☁️ Thống kê độ mây:")
            print(f"   - Trung bình: {np.mean(cloud_stats):.1f}%")
            print(f"   - Tối thiểu: {np.min(cloud_stats):.1f}%")
            print(f"   - Tối đa: {np.max(cloud_stats):.1f}%")
            print(f"   - Ngưỡng đã đặt: {CLOUD_COVER_THRESHOLD}%")
            
            # Phân phối theo khoảng
            low_cloud = sum(1 for c in cloud_stats if c < 10)
            medium_cloud = sum(1 for c in cloud_stats if 10 <= c < 30)
            high_cloud = sum(1 for c in cloud_stats if c >= 30)
            
            print(f"   - Ít mây (<10%): {low_cloud} ảnh")
            print(f"   - Trung bình (10-30%): {medium_cloud} ảnh")
            print(f"   - Nhiều mây (>30%): {high_cloud} ảnh")
        
    except Exception as e:
        print(f"❌ Không thể lấy thống kê mây: {str(e)}")

def clean_output_folder():
    """Xóa tất cả file trong thư mục output"""
    if not os.path.exists(output_folder):
        print("❌ Thư mục không tồn tại")
        return
    
    files = [f for f in os.listdir(output_folder) if f.endswith('.tif')]
    
    if not files:
        print("📂 Thư mục đã trống")
        return
    
    total_size = sum(os.path.getsize(os.path.join(output_folder, f)) for f in files) / (1024*1024)
    
    confirm = input(f"⚠️  Xóa {len(files)} file ({total_size:.1f} MB)? (y/N): ")
    if confirm.lower() == 'y':
        for file in files:
            os.remove(os.path.join(output_folder, file))
        print(f"🗑️  Đã xóa {len(files)} file")
    else:
        print("❌ Hủy bỏ")

# Kiểm tra kết quả hiện tại
files = check_downloaded_files()

if files:
    print(f"\n💡 Các function hữu ích:")
    print(f"   - analyze_temporal_coverage(): Phân tích độ phủ thời gian")
    print(f"   - view_file_info('filename.tif'): Xem chi tiết file")
    print(f"   - get_cloud_statistics(): Thống kê độ mây")
    print(f"   - clean_output_folder(): Xóa tất cả file")
    
    # Tự động phân tích độ phủ thời gian
    print(f"\n📊 Phân tích độ phủ thời gian:")
    analyze_temporal_coverage()
    
    # Thống kê độ mây
    print(f"\n")
    get_cloud_statistics()
    
    # Hiển thị thông tin file đầu tiên
    if len(files) > 0:
        print(f"\n🔍 Thông tin file đầu tiên:")
print(f"   - Phân tích NDVI, NDWI, LSWI để đánh giá độ ẩm đất")

print(f"\n🎯 Dữ liệu Sentinel-2 phù hợp cho:")
print(f"   - Phân tích NDVI, NDMI để đánh giá độ ẩm đất")

print(f"   - Chỉ 4 chỉ số cần thiết - file nhỏ gọn hơn")print(f"   - Tích hợp với dữ liệu Sentinel-1 và SMAP")
print(f"   - Lập bản đồ độ ẩm đất nông nghiệp")
print(f"   - Theo dõi thay đổi theo thời gian")